Google Colab Setup

In [ ]:
!git clone https://github.com/jan1na/Bleeding-Detecting-on-Capsule-Endoscopy.git

import sys

sys.path.append('/content/Bleeding-Detecting-on-Capsule-Endoscopy/scripts')


Drive Setup

In [ ]:
from google.colab import drive

# Mount Google Drive (for data storage and access to dataset)
drive.mount('/content/drive')

# Deep Learning Part

## Imports

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torch.optim import lr_scheduler
from torch.utils.data.sampler import WeightedRandomSampler

from models import MobileNetV2
from bleeding_dataset import BleedDataset

## Training Space

## Model Training Configuration

This section defines the key hyperparameters and settings for training the model.

- **SAVE_PATH:** Directory where model checkpoints and training results will be saved.
- **TRAIN_TEST_SPLIT:** Specifies the ratio for training and validation data. The remaining portion will be used for testing.
- **DIRECTORY_PATH:** Location of the dataset used for training and evaluation.
- **BATCH_SIZE:** Number of samples processed in each training step.
- **LR (Learning Rate):** The step size used by the optimizer to update model weights.
- **NUM_OF_EPOCHS:** Total number of training iterations over the dataset.
- **EARLY_STOP_LIMIT:** The number of epochs without improvement before stopping training.
- **THRESHOLD:** Classification threshold for predicting bleeding images.
- **MODEL:** Specifies which deep learning architecture to use (MobileNetV2, ResNet, AlexNet, or VGG19).
- **APPLY_AUGMENTATION:** Determines whether to apply data augmentation to handle class imbalance.
- **USE_COSINE_ANNEALING_LR:** Enables cosine annealing for learning rate adjustment.
- **AUGMENT_TIMES:** Number of times each image is augmented during training.

Adjust these parameters as needed to optimize model performance.


### Hyperparameters

In [ ]:
# Path to save model checkpoints and results
SAVE_PATH = "/content/drive/MyDrive/Colab Notebooks/DL4MI/runs/"

# Train/test split ratio
TRAIN_TEST_SPLIT = (0.8, 0.1)  

# Dataset directory path
DIRECTORY_PATH = "/content/drive/MyDrive/Colab Notebooks/DL4MI/project_capsule_dataset"

# Training parameters
BATCH_SIZE = 16
LR = 0.001
NUM_OF_EPOCHS = 20
EARLY_STOP_LIMIT = 3
THRESHOLD = 0.5  

# Model selection
MODEL = MobileNetV2  

# Training settings
APPLY_AUGMENTATION = False
USE_COSINE_ANNEALING_LR = True
AUGMENT_TIMES = 8


### Dataset, Model etc. Inıtıalization

## Model Initialization and Dataset Preparation

This section initializes the model, prepares the dataset, and configures the training parameters.

- **Device Selection:** Automatically selects CUDA if available, otherwise defaults to CPU.
- **Model Initialization:** Loads the selected model architecture and assigns a unique serial number for saving checkpoints.
- **Dataset Preparation:** 
  - Splits the dataset into training, validation, and test sets based on predefined ratios.
  - Applies optional augmentation to balance class distribution.
  - Uses a **WeightedRandomSampler** to address class imbalance.
- **Loss Function:** Uses **BCEWithLogitsLoss** with class weighting to ensure bleeding images are properly accounted for.
- **Optimizer and Scheduler:** 
  - Uses **Adam** optimizer for efficient parameter updates.
  - Supports either **Cosine Annealing LR** or **StepLR** scheduling for dynamic learning rate adjustment.


In [ ]:
# Set device (CUDA if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()

# Model Initialization
def initialize_model(model_class, save_path: str):
    """
    Initializes the model and creates a directory for saving checkpoints.

    :param model_class: Model architecture to be used (MobileNetV2, ResNet, etc.)
    :param save_path: Directory where model checkpoints will be saved.
    :return: Initialized model and save path.
    """
    model = model_class().to(device)
    model_serial_number = f"training_with_{model.__class__.__name__}_{datetime.now().strftime('on_%m.%d._at_%H:%M:%S')}"
    model_serial_path = os.path.join(save_path, model_serial_number)
    os.makedirs(model_serial_path, exist_ok=True)

    return model, model_serial_path

model, model_serial_path = initialize_model(MODEL, SAVE_PATH)
full_dataset = None  # Placeholder for dataset


# Dataset Preparation
def prepare_datasets(dataset_class, directory: str, split_ratios: list[int], batch_size: int, image_mode: str = "RGB", seed: int = 0):
    """
    Prepares train, validation, and test datasets.

    :param dataset_class: Dataset class to be used (e.g., BleedDataset)
    :param directory: Path to dataset directory
    :param split_ratios: Ratios for train, validation, and test splits
    :param batch_size: Number of samples per batch
    :param image_mode: Image format ("RGB" or "gray")
    :param seed: Random seed for reproducibility
    :return: Dictionary containing DataLoaders for train, validation, and test sets
    """
    global full_dataset
    full_dataset = dataset_class(directory, mode=image_mode, apply_augmentation=APPLY_AUGMENTATION, augment_times=AUGMENT_TIMES)
    
    total_size = len(full_dataset)
    train_size = int(split_ratios[0] * total_size)
    test_size = int(split_ratios[1] * total_size)
    validation_size = total_size - train_size - test_size

    torch.manual_seed(seed)
    train_dataset, validation_dataset, test_dataset = random_split(full_dataset, [train_size, validation_size, test_size])

    train_labels = np.array(full_dataset.get_labels())[train_dataset.indices]
    class_counts = np.bincount(train_labels)
    class_weights = 1.0 / class_counts
    weights = [class_weights[label] for label in train_labels]
    sampler = WeightedRandomSampler(weights, len(weights))

    return {
        "train": DataLoader(train_dataset, batch_size=batch_size, num_workers=4, pin_memory=True, sampler=sampler),
        "validation": DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True),
        "test": DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True),
    }

data_loaders = prepare_datasets(BleedDataset, DIRECTORY_PATH, TRAIN_TEST_SPLIT, BATCH_SIZE, image_mode="RGB", seed=0)

# Class Weight Calculation for Imbalance Handling
bleeding_weight = 6161 / (713 * AUGMENT_TIMES) if APPLY_AUGMENTATION else 6161 / 713
weights = torch.tensor([1.0, bleeding_weight]).to(device)

# Loss Function
criterion = nn.BCEWithLogitsLoss(pos_weight=weights[1])

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# Learning Rate Scheduler
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_OF_EPOCHS) if USE_COSINE_ANNEALING_LR else lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


### Training Loop

## Model Training Process

This section implements the full training loop, including validation and early stopping.

### Training Workflow:
- **Data Augmentation:** Enabled during training, disabled for validation.
- **Training Loop:**
  - Iterates over training batches, computes predictions, and updates weights.
  - Uses **Binary Cross Entropy with Logits Loss (BCEWithLogitsLoss)** as the loss function.
- **Validation Loop:**
  - Runs without gradient calculations to evaluate model performance.
  - Computes validation loss to track progress.
- **Checkpointing:**
  - Saves the model whenever a new minimum validation loss is achieved.
  - Deletes the previous best checkpoint before saving a new one.
- **Early Stopping:**
  - Monitors validation loss and stops training if no improvement occurs for `EARLY_STOP_LIMIT` epochs.
- **Learning Rate Adjustment:**
  - The learning rate is updated according to the selected scheduler (Cosine Annealing or StepLR).

This ensures efficient training while preventing overfitting.


In [ ]:
# Training Initialization
train_losses, validation_losses = [], []
min_validation_loss, min_validation_path = None, None
early_stop_step = 0  

# Training Loop
for epoch in range(NUM_OF_EPOCHS):
    if APPLY_AUGMENTATION:
        full_dataset.enable_augmentation()

    averaged_training_loss = 0  

    for batch_idx, (images, labels) in tqdm(enumerate(data_loaders['train']), leave=False):
        images, labels = images.to(device), labels.to(device)  
        
        model.train()
        outputs = model(images.type(torch.float))

        float_outputs = outputs[:, 0].type(torch.float)
        float_labels = labels.type(torch.float)

        train_loss = criterion(float_outputs, float_labels)
        averaged_training_loss += train_loss  

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

    averaged_training_loss /= len(data_loaders['train'])

    # Validation Loop
    with torch.no_grad():
        if APPLY_AUGMENTATION:
            full_dataset.disable_augmentation()

        model.eval()
        validation_loss = 0.0

        for validation_images, validation_labels in data_loaders['validation']:
            validation_images, validation_labels = validation_images.to(device), validation_labels.to(device)
            validation_outputs = model(validation_images.type(torch.float))

            float_validation_outputs = validation_outputs[:, 0].type(torch.float)
            float_validation_labels = validation_labels.type(torch.float)

            validation_loss += criterion(float_validation_outputs, float_validation_labels).item()

        validation_loss /= len(data_loaders['validation'])

    # Model Checkpointing
    if min_validation_loss is None or validation_loss < min_validation_loss:
        early_stop_step = 0
        min_validation_loss = validation_loss  

        if min_validation_path:
            os.remove(min_validation_path)

        min_validation_path = os.path.join(model_serial_path, f"min_validation_loss:{min_validation_loss}_epoch:{epoch}.pth")
        torch.save(model, min_validation_path)
    else:
        early_stop_step += 1

    print(f"Epoch: {epoch + 1} | training loss: {averaged_training_loss.item()} | min validation loss: {min_validation_loss}", flush=True)
    
    train_losses.append(averaged_training_loss.item())
    validation_losses.append(validation_loss)

    scheduler.step()

    if early_stop_step >= EARLY_STOP_LIMIT:
        print("early stopping...")
        break


In [ ]:
plt.plot(train_losses, color='blue', label='Train Loss')
plt.plot(validation_losses, color='orange', label='Validation Loss')
plt.legend()
plt.savefig(os.path.join(model_serial_path, "losses.png"))

## Testing Space

## Model Testing and Evaluation

This section loads the best-performing model and evaluates its accuracy on the test set.

### Testing Workflow:
- **Load Best Model:** The checkpoint with the lowest validation loss is loaded.
- **Disable Augmentation:** Ensures consistency in test data.
- **Inference:**
  - Runs the model on test images in evaluation mode.
  - Applies the classification threshold to determine predictions.
- **Performance Calculation:**
  - Tracks correct predictions separately for each class (bleeding vs. healthy).
  - Computes the total number of samples per class.

This step provides a final assessment of model performance on unseen data.


In [ ]:
# Load best model and set to evaluation mode
loaded_model = torch.load(min_validation_path)
loaded_model = loaded_model.eval()

# Initialize counters for correct predictions per class
class_correct, class_total = [0, 0], [0, 0]

with torch.no_grad():
    if APPLY_AUGMENTATION:
        full_dataset.disable_augmentation()

    for test_images, test_labels in tqdm(data_loaders['test']):
        test_images, test_labels = test_images.to(device), test_labels.to(device)

        test_outputs = loaded_model(test_images.type(torch.float))
        test_outputs = test_outputs.squeeze().type(torch.float)

        test_outputs[test_outputs >= THRESHOLD] = 1
        test_outputs[test_outputs < THRESHOLD] = 0

        correct = (test_outputs == test_labels).squeeze()

        for e, label in enumerate(test_labels):
            class_correct[label] += correct[e].item()
            class_total[label] += 1


## Model Performance Evaluation

This section calculates and displays the model's accuracy on the test dataset.

### Accuracy Metrics:
- **Total Accuracy:** Overall percentage of correctly classified images.
- **Healthy Detection Accuracy:** The proportion of correctly classified healthy images.
- **Bleeding Detection Accuracy:** The proportion of correctly classified bleeding images.

### Saving Results:
- The accuracy metrics are saved in a text file inside the model's directory for future reference.

### GPU Memory Management:
- Clears the GPU memory cache after evaluation to free up resources.


In [ ]:
# Compute and print accuracy metrics
print(f"Total accuracy: {sum(class_correct) / sum(class_total)} on threshold: {THRESHOLD}")
print(f"Healthy detection: {class_correct[0]}/{class_total[0]} | accuracy: {class_correct[0] / class_total[0]}")
print(f"Bleeding detection: {class_correct[1]}/{class_total[1]} | accuracy: {class_correct[1] / class_total[1]}")

# Save accuracy results to a text file
with open(os.path.join(model_serial_path, "accuracy.txt"), 'w') as txt:
    txt.write(f"Total accuracy: {sum(class_correct) / sum(class_total)} on threshold: {THRESHOLD}\n")
    txt.write(f"Healthy detection: {class_correct[0]}/{class_total[0]} | accuracy: {class_correct[0] / class_total[0]}\n")
    txt.write(f"Bleeding detection: {class_correct[1]}/{class_total[1]} | accuracy: {class_correct[1] / class_total[1]}\n")

# Clear GPU memory cache after evaluation
torch.cuda.empty_cache()
